In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
from PIL import Image
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing
import multiprocessing.dummy as mp
from utils import *
%matplotlib inline


DATA_DIR = '/data/amiratag/prostate'
MEM_DIR = '/home/amiratag/Projects/Prostate'

In [2]:
train_csv = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
gleason_scores = train_csv['gleason_score'][train_csv['data_provider'] == 'radboud'].values
valid_images = train_csv['image_id'][train_csv['data_provider'] == 'radboud'][:4160] #Rest for test
image_paths = [os.path.join(DATA_DIR, 'train_images', i)+'.tiff' for i in valid_images
               if os.path.exists(os.path.join(DATA_DIR, 'train_images', i)+'.tiff')]
mask_paths = [i.replace('train_images','train_label_masks').replace('.tiff', '_mask.tiff') for i in image_paths]

In [7]:
def create_patches(i, image_paths, mask_paths, im_shape):
    
    try:
        slide = io.imread(image_paths[i])
        mask = io.imread(mask_paths[i])
    except:
        return np.zeros((0, 2)).astype(str)
    assert slide.shape == mask.shape
    size = slide.shape
    names, y = [], []
    for w in range(size[0] // im_shape[0]):
        for h in range(size[1] // im_shape[1]):
            msk = mask[w * im_shape[0]: (w+1) * im_shape[0], h * im_shape[1]: (h+1) * im_shape[1]][..., 0]
            for score in np.arange(5, -1, -1):
                if np.mean(np.equal(msk, score)) > 0.025:
                    break
            y.append(score)
            if y[-1] == 0 and np.random.random() > 0.2:
                # We don't want to save all background tiles as there are too many
                continue
            slide_name = image_paths[i].split('/')[-1].split('.')[0]
            patch = slide[w * im_shape[0]: (w+1) * im_shape[0], h * im_shape[1]: (h+1) * im_shape[1]]
            patch_name = '{}_{}_{}.png'.format(slide_name, str(w).zfill(3), str(h).zfill(3))
            names.append(patch_name)
            Image.fromarray(patch).save(os.path.join(DATA_DIR, 'patches', str(y[-1]), patch_name))
    return np.stack([names, y], -1)

Create Patches

In [8]:
n_cpus = multiprocessing.cpu_count()
pool = mp.Pool(n_cpus)
df = np.zeros((0, 2)).astype(str)
for n in tqdm(range(int(np.ceil(len(image_paths) / n_cpus)))):
    output = pool.map(lambda i: create_patches(i, image_paths, mask_paths, (224, 224)),
                      np.arange(n_cpus*n, n_cpus*(n+1)))
    df = np.concatenate([df, np.concatenate(output)])
    pd.DataFrame(df, columns=['patch', 'label']).to_csv(os.path.join(DATA_DIR, 'labels.csv'))


  0%|          | 0/130 [00:00<?, ?it/s]

ValueError: all input arrays must have the same shape